In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

from elasticsearch import Elasticsearch
from datasets import load_dataset


from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

from elasticsearch import helpers
from tqdm import tqdm

# Load single PDF file
def load_single_pdf(file_path):
    loader = PyPDFLoader(file_path)
    pages = loader.load()
    return pages

# Load multiple PDF files from directory
def load_pdf_directory(directory_path):
    loader = PyPDFDirectoryLoader(directory_path)
    pages = loader.load()
    return pages



In [2]:
pdf_paths = "../data/pdf/"

In [3]:
pdf_data = load_pdf_directory(pdf_paths)

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # PDF 문서는 더 작은 청크로 나누는 것이 좋음
    chunk_overlap=50, # 청크 간 중복도 줄임
    length_function=len,
    separators=["\n\n", "\n", ".", "!", "?", ";", ":", " ", ""],  # PDF 문서의 구조를 고려한 구분자 추가
    is_separator_regex=False
)

chunks = text_splitter.split_documents(pdf_data)

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="../ai_models/base_models/BGE-m3-ko",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
es = Elasticsearch("http://127.0.0.1:9200")


In [6]:
es.options(ignore_status=400).indices.create(
    index="pdf",
    mappings={
        "properties": {
            "content": {"type": "text"},
            "embedding": {
                "type": "dense_vector",
                "dims": 1024,  # 모델에 따라 조정
                "index": True,
                "similarity": "cosine"
            }
        }
    }
)

ObjectApiResponse({'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [pdf/nNfCzCNMQrW82xV-II5ouw] already exists', 'index_uuid': 'nNfCzCNMQrW82xV-II5ouw', 'index': 'pdf'}], 'type': 'resource_already_exists_exception', 'reason': 'index [pdf/nNfCzCNMQrW82xV-II5ouw] already exists', 'index_uuid': 'nNfCzCNMQrW82xV-II5ouw', 'index': 'pdf'}, 'status': 400})

In [7]:
actions = []
for chunk in tqdm(chunks):
    text = chunk.page_content
    vector = embeddings.embed_documents([text])[0]  # ✅ 문서용

    actions.append({
        "_index": "pdf",
        "_source": {
            "content": text,
            "embedding": vector
        }
    })

helpers.bulk(es, actions)

100%|██████████| 13590/13590 [2:28:05<00:00,  1.53it/s] 


(13590, [])

In [ ]:
query = "피지컬컬 AI 설명"
query_vector = embeddings.embed_query(query)

response = es.search(
    index="pdf",
    knn={
        "field": "embedding",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 20
    }
)

# 결과 출력
for hit in response["hits"]["hits"]:
    print(f"점수: {hit['_score']:.4f}")
    print(f"문서 내용: {hit['_source']['content']}\n")


점수: 0.8313
문서 내용: <표 2-1> 피지컬 AI 정의 요약
* 출처: 소프트웨어정책연구소 정리(2025.04.01.)■현재까지 피지컬 AI에 대한 통일된 정의는 부재한 상황이나, 산·학·연 각 분야에서는 공통적으로 ‘AI의 물리적 구현’, ‘물리적 인터페이스를 통한 실제 세계와의 상호작용’, ‘자율적 판단·행동’ 등을 핵심 요소로 강조●본 보고서에서는 이러한 핵심 요소를 바탕으로 피지컬 AI를 아래와 같이 정의하고  주요 기술과 유형을 분석AI가 물리적 실체 안에 구현되어 센서와 액추에이터 등을 통해 현실 세계를 인식하고, 자율적으로 판단·행동함으로써 환경과 유기적으로 상호작용할 수 있는 시스템피지컬 AI 정의

점수: 0.8273
문서 내용: <표 2-2> 피지컬 AI 유형 구분
* 출처: 소프트웨어정책연구소 정리(2025.04.01.)

점수: 0.8255
문서 내용: 피지컬 AI를 정의하고, 그 실현을 위한 관련 기술 개발에 박차●NVIDIA는 피지컬 AI를 “현실(물리적) 세계에서 복잡한 행동을 인식, 이해 및 수행할 수 있는 자율 시스템(로봇, 자율주행차, 스마트 공간 등)”으로 설명(NVIDIA, 2025e)s∙ 피지컬 AI는 물리 세계에 대한 통찰을 생성하고 실행할 수 있다는 점에서 ‘생성 피지컬 AI(Generative Physical AI)’로도 불리며, 이는 기존 생성 AI를 확장해 3D 세계의 공간적 관계와 물리적 행동을 학습하고, 단순한 시각 재현을 넘어 현실의 물리 법칙을 반영하는 시스템∙ 과 거  자 율  기 계 가  주 변  환 경  인 지 와  상 호 작 용 에  한 계 가  있 었 던  반 면, 피 지 컬  AI는  현 실  세 계 와 의  자 연 스 러 운  상 호 작 용 을 가능하게 함으로써, 복잡한 작업 수행 능력을 높이고 인간과의 협업을 더욱 효율적이고 직관적으로 만드는 데 기여●구글의

점수: 0.8218
문서 내용: . et al., 2021)의 연구에서 피지컬 AI는 디지털 AI의 확장 개념으로, “물리적 